# TALN Project

Installing BERT

In [0]:
#!pip install bert-embedding
import numpy

Reading the Datafiles

In [0]:
!unrar x /content/Corpus-PADT-txt.rar



UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /content/Corpus-PADT-txt.rar

Creating    Corpus-PADT-txt                                           OK
Extracting  Corpus-PADT-txt/ALH20010911.00010_stor.txt                     0%  OK 
Extracting  Corpus-PADT-txt/ALH20010911.0001_story.txt                     0%  OK 
Extracting  Corpus-PADT-txt/ALH20010911.0002_story.txt                     1%  OK 
Extracting  Corpus-PADT-txt/ALH20010911.0003_story.txt                     2%  OK 
Extracting  Corpus-PADT-txt/ALH20010911.0004_story.txt                     3%  OK 
Extracting  Corpus-PADT-txt/ALH20010911.0005_story.txt                     6%  OK 
Extracting  Corpus-PADT-txt/ALH20010911.0006_story.txt                     7%  OK 
Extracting  Corpus-PADT-txt/ALH20010911.0007_story.txt                     9%  OK 
Extracting  Corpus-PADT-txt/ALH20010911.0008_story.txt                   

In [27]:
corpus=[]
f=open("filenames.txt")
for i in f:
    content=open("Corpus-PADT-txt/"+i[:-1],encoding='utf-8')
    corpus.append(content.read())
print(corpus)

['', '\nفؤاد كنعان، رائد القصة القصيرة في لبنان رحل مساء أمس عن 81 عاماً.\n\nأستاذ من أساتذة النثر العربي المضيء، المشبع بالجمالية والأناقة اللفظية والعمق الوجودي. قاص كان له أثره الكبير في تأسيس الشكل الجديد للقصة القصيرة في لبنان، وقبل ان يجدد في حقل الفن القصصي كان خير وارث لجيل النهضويين، لغة وأدباً.\n\nكتابه القصصي الأول الذي صدر في العام 1947 وضع مقدمته »شيخ الأدباء« مارون عبود، وكانت بمثابة المقدمة الوحيدة التي لم يكتب مارون عبود سواها لأي كتاب، أياً كان صاحبه.\n\nولعل شغف فؤاد كنعان اللغوي ونزعته الى الاناقة في التعبير، والمتانة في السبك، جعلاه من الكتّاب المقلين، الذين يرفضون الاطناب والافاضة، وكان يؤثر التمهل في الكتابة متخيراً ألفاظه بدقة وجمالية، وصائغاً جمله صوغ الأساتذة الماهرين. مجموعته القصصية »أولاً وآخراً وبين بين« كانت أشبه بالحدث القصصي عندما صدرت في العام 1974. وعرفت تلك القصص بجرأة نادرة راح الكاتب يواجه من خلالها موضوعات كانت شائكة ولا تزال كالجنس والخيانة.\n\nوله في حقل الترجمة أعمال عدة من أبرزها رواية الكاتب الفرنسي بلزاك »أوجني غرانده« وقيل انها من أفضل الترج

Data prepocessing(Removing stopword, Tokenizing, Stemming) 

In [28]:
import nltk
nltk.download('stopwords')
from nltk import stem
from nltk.corpus import stopwords
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
stopwords_list = stopwords.words('arabic')
arabicstem =stem.snowball.ArabicStemmer()

In [0]:
def preprocess(data):
    data = data.split('.')
    tokens = []
    for sentence in data:
        tokens.append(nltk.word_tokenize(sentence))
    tokens_updated=[]
    for token in tokens:
        sentence=[]
        for word in token:
            if word.isalpha():                  
                sentence.append(word)
        tokens_updated.append(sentence)
    
    for (indx,sentence) in enumerate(tokens_updated):
        
        for word in sentence:
            if word in stopwords_list:
                tokens_updated[indx].remove(word)  
    tokens_updated = [ l for l in tokens_updated if l !=[]]
    return tokens_updated 
new_corpus = []
for data in corpus:
    new_corpus.extend(preprocess(data))

Extracting all valide bigrams from the corpus

In [0]:
from nltk.util import ngrams
from collections import Counter
bigrams = []
new_bigram=[]
for token in new_corpus:
    bigrams.append(Counter(ngrams(token,2)))
for i in range(len(bigrams)) :
    
    new_bigram.append([[x,y] for x,y in  bigrams[i].items()])


Sort the bigrams according to occurance

In [0]:
import itertools
sortedbigrams = list(itertools.chain(* new_bigram))
sortedbigrams =sorted(sortedbigrams, key = lambda x: x[1], reverse=True)

Some statics

In [37]:
count3=0
count2=0
count1=0
for i in sortedbigrams:
    if i[1]==3:
        count3+=1
    elif i[1]==2:
        count2+=1
    elif i[1]==1:
        count1+=1
print("Bigrams occured 3 times :"+str(count3))
print("Bigrams occured 2 times :"+str(count2))
print("Bigrams occured one time :"+str(count1))

Bigrams occured 3 times :6
Bigrams occured 2 times :130
Bigrams occured one time :38257


BERT Model

In [40]:
from bert_embedding import BertEmbedding
bert_embedding = BertEmbedding(model='bert_12_768_12', dataset_name='wiki_multilingual')


/usr/local/lib/python3.6/dist-packages/gluonnlp/model/bert.py:693: UserWarning: wiki_cn/wiki_multilingual will be deprecated. Please use wiki_cn_cased/wiki_multilingual_uncased instead.
  warnings.warn('wiki_cn/wiki_multilingual will be deprecated.'


Vocab file is not found. Downloading.


extracting thresholds for collocations

In [0]:

from sklearn.metrics.pairwise import cosine_similarity
def calcul_cosine_BERT(word1,word2):
  wordembedding=bert_embedding([word1,word2])
  return(cosine_similarity(wordembedding[0][1], wordembedding[1][1]))


In [164]:
calcul_cosine_BERT(sortedbigrams[0][0][0],sortedbigrams[0][0][1])[0][0]

0.6954264

In [0]:
import pandas as pd
dataframe=pd.DataFrame(columns=["words","cosine_bert"])

In [0]:
#calculate all the cosine similarity od all valid bigrams
dataframe["words"]=[(sortedbigrams[i][0][0],sortedbigrams[i][0][1])  for i in range(len(sortedbigrams)) if sortedbigrams[i][1]>1 ]
dataframe["cosine_bert"]=[calcul_cosine_BERT(sortedbigrams[i][0][0],sortedbigrams[i][0][1])[0][0]  for i in range(len(sortedbigrams)) if sortedbigrams[i][1]>1 ]

In [0]:
threshlodBert=dataframe.quantile([.25,.75],axis=0)
minBert=list(threshlodBert.cosine_bert)[0]
maxBert=list(threshlodBert.cosine_bert)[1]

In [0]:
#classification function
def classify_collection_BERT(test1,test2):
  if minBert<= calcul_cosine_BERT(test1,test2)<= maxBert:
    print('#Oui')
  else:
    print('#Non')


In [122]:
classify_collection('تسلق','الجبال')

#Oui


Word2Vec

In [0]:
feature_size = 100      
window_context = 30                                                                                           
min_word_count = 1                           
sample = 1e-3   
from gensim.models import Word2Vec
model = Word2Vec(new_corpus,  size=feature_size,  window=window_context, min_count=min_word_count,sample=sample, iter=50)


In [218]:
model.wv.vocab

{'فؤاد': <gensim.models.keyedvectors.Vocab at 0x7f56e1f96fd0>,
 'رائد': <gensim.models.keyedvectors.Vocab at 0x7f56e1f96a90>,
 'القصة': <gensim.models.keyedvectors.Vocab at 0x7f56e1f96cc0>,
 'القصيرة': <gensim.models.keyedvectors.Vocab at 0x7f56e1f96e48>,
 'لبنان': <gensim.models.keyedvectors.Vocab at 0x7f56e1f96a20>,
 'رحل': <gensim.models.keyedvectors.Vocab at 0x7f56e1f96b00>,
 'مساء': <gensim.models.keyedvectors.Vocab at 0x7f56e1f96d30>,
 'أمس': <gensim.models.keyedvectors.Vocab at 0x7f56e1f96dd8>,
 'أستاذ': <gensim.models.keyedvectors.Vocab at 0x7f56e1f96d68>,
 'أساتذة': <gensim.models.keyedvectors.Vocab at 0x7f56e1f96ef0>,
 'النثر': <gensim.models.keyedvectors.Vocab at 0x7f56e1f96c88>,
 'العربي': <gensim.models.keyedvectors.Vocab at 0x7f56e1f96978>,
 'المشبع': <gensim.models.keyedvectors.Vocab at 0x7f56e1f96eb8>,
 'بالجمالية': <gensim.models.keyedvectors.Vocab at 0x7f56e1f96cf8>,
 'والأناقة': <gensim.models.keyedvectors.Vocab at 0x7f56e1f96f60>,
 'اللفظية': <gensim.models.keyedvec

In [0]:
def calcul_cosine_W2V(word1,word2):
  try:
    cosine_similarity = numpy.dot(model[word1], model[word2])/(numpy.linalg.norm(model[word1])* numpy.linalg.norm(model[word2]))
    return(cosine_similarity)
  except:
    print("Words dosen't exist int the vocabluray")

In [220]:
dataframe['cosine_W2V']=[calcul_cosine_W2V(sortedbigrams[i][0][0],sortedbigrams[i][0][1])  for i in range(len(sortedbigrams)) if sortedbigrams[i][1]>1 ]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [221]:
dataframe.head()

,words,cosine,cosine_W2V,cosine_FastText,cosine_bert
0,"(التأمين, الطبي)",0.695426,0.990894,0.999998,0.695426
1,"(الولايات, المتحدة)",0.731942,0.925692,0.999989,0.731942
2,"(التجارة, العالمية)",0.506086,0.918413,0.999999,0.506086
3,"(بليون, دولار)",0.722890,0.964351,0.999961,0.722890
4,"(المئة, ومؤشر)",1.000000,0.917558,0.999708,1.000000


In [0]:
threshlodW2V=dataframe.quantile([.25,.75],axis=0)
minW2V=list(threshlodW2V.cosine_W2V)[0]
maxW2V=list(threshlodW2V.cosine_W2V)[1]

In [0]:
#classification function
def classify_collection_W2V(test1,test2):
  try:
    if minW2V < calcul_cosine_W2V(test1,test2)< maxW2V:
      print('#Oui')
    else:
      print('#Non')
  except:
    return(0)

In [193]:

classify_collection_W2V('تسلق','الجبال')

Words dosen't exist int the vocabluray


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


0

Fasttext

In [197]:
from gensim.models import FastText
model_Fasttext = FastText(new_corpus, min_count=1)

In [0]:
def calcul_cosine_FastText(word1,word2):
  try:
    cosine_similarity = numpy.dot(model_Fasttext[word1], model_Fasttext[word2])/(numpy.linalg.norm(model_Fasttext[word1])* numpy.linalg.norm(model_Fasttext[word2]))
    return(cosine_similarity)
  except:
    print("Words dosen't exist int the vocabluray")

In [199]:
dataframe['cosine_FastText']=[calcul_cosine_FastText(sortedbigrams[i][0][0],sortedbigrams[i][0][1])  for i in range(len(sortedbigrams)) if sortedbigrams[i][1]>1 ]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
threshlodFT=dataframe.quantile([.25,.75],axis=0)
minFT=list(threshlodFT.cosine_FastText)[0]
maxFT=list(threshlodFT.cosine_FastText)[1]

In [0]:
#classification function
def classify_collection_FastText(test1,test2):
  try:
    if minFT < calcul_cosine_FastText(test1,test2)< maxFT:
      print('#Oui')
    else:
      print('#Non')
  except:
    return(0)

In [225]:
dataframe

,words,cosine_W2V,cosine_FastText,cosine_bert
0,"(التأمين, الطبي)",0.990894,0.999998,0.695426
1,"(الولايات, المتحدة)",0.925692,0.999989,0.731942
2,"(التجارة, العالمية)",0.918413,0.999999,0.506086
3,"(بليون, دولار)",0.964351,0.999961,0.722890
4,"(المئة, ومؤشر)",0.917558,0.999708,1.000000
...,...,...,...,...
132,"(بليون, دولار)",0.964351,0.999961,0.722890
133,"(مستثمرين, عرب)",0.876029,0.999918,0.116368
134,"(في, المئة)",0.853919,0.999945,0.884881
135,"(السياسة, الاميركية)",0.856015,0.999999,0.727116


In [212]:
classify_collection_BERT('الجماعات', 'المسلحة')		

#Non


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until
